In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark import SparkConf
import gc


print("loaded libraries")
spark = SparkSession.builder \
        .appName("worst labs") \
        .getOrCreate()

loaded libraries


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/01 14:08:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/01 14:08:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
labs = spark.read.parquet("/project2/wparker/SIPA_data/RCLIF_labs_10312023.parquet")
labs = labs.withColumn('lab_result_time',f.to_timestamp('lab_order_time','yyyy-MM-dd HH:mm:ss'))
labs = labs.select('C19_HAR_ID', 'lab_result_time','lab_name', 'lab_value')

select_expr = [f.regexp_replace(f.col('lab_name'), "[\ufeff]", "").alias('lab_name')]
labs = labs.select('C19_HAR_ID', 'lab_result_time', 'lab_value', *select_expr)

select_expr = [f.regexp_replace(f.col('lab_value'), "[\ufeff]", "").alias('lab_value')]
labs = labs.select('C19_HAR_ID', 'lab_result_time', 'lab_name', *select_expr)

select_expr = [f.regexp_replace(f.col('lab_value'), "[<]", "").alias('lab_value')]
labs = labs.select('C19_HAR_ID', 'lab_result_time', 'lab_name', *select_expr)

select_expr = [f.regexp_replace(f.col('lab_value'), "[>]", "").alias('lab_value')]
labs = labs.select('C19_HAR_ID', 'lab_result_time', 'lab_name', *select_expr)
labs = labs.withColumn("hourly_timestamp", f.date_trunc("hour", labs.lab_result_time))

In [3]:
cohort = spark.read.parquet("/project2/wparker/SIPA_data/life_support_cohort.parquet")
cohort = cohort.withColumn('life_support_start_time',f.to_timestamp('life_support_start','yyyy-MM-dd HH:mm:ss'))
cohort = cohort.select('C19_HAR_ID', 'life_support_start_time')

cohort = cohort.withColumn('window_start', (f.col('life_support_start_time')-f.expr("INTERVAL 42 HOURS")))
cohort = cohort.withColumn('window_end', (f.col('life_support_start_time')+f.expr("INTERVAL 6 HOURS")))
cohort_hours = cohort.withColumn('time', f.explode(f.expr('sequence(window_start, window_end, interval 1 hour)')))
cohort_hours = cohort_hours.withColumn('time',f.to_timestamp('time','yyyy-MM-dd HH:mm:ss'))

cohort_hours = cohort_hours.withColumn("hourly_timestamp", f.date_trunc("hour", cohort_hours.time))
cohort_hours = cohort_hours.select('C19_HAR_ID', 'life_support_start_time', 'hourly_timestamp')

In [4]:
group_cols = ["C19_HAR_ID","hourly_timestamp"]
cohort_labs_48 = cohort_hours.join(labs, on=group_cols, how="left")

cohort_labs_48 = cohort_labs_48.select('C19_HAR_ID', 'life_support_start_time','hourly_timestamp', 
                                       'lab_name', 'lab_value')
cohort_labs_48 = cohort_labs_48.withColumn("lab_value_num",cohort_labs_48.lab_value.cast('double'))
cohort_labs_48 = cohort_labs_48.filter(f.col('lab_name').isNotNull())

In [5]:
group_cols = ["C19_HAR_ID", "life_support_start_time","hourly_timestamp"]
cohort_labs_wide = cohort_labs_48.groupBy(group_cols) \
                                     .pivot("lab_name") \
                                     .agg(f.min('lab_value_num').alias("min"),
                                         f.max('lab_value_num').alias("max")).orderBy(group_cols)
cohort_labs_wide.write.parquet("/project2/wparker/SIPA_data/cohort_labs_48.parquet", mode="overwrite")

23/11/01 14:09:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
group_cols = ["C19_HAR_ID", "life_support_start_time"]
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
cohort_labs_wide_summary = cohort_labs_48.groupBy(group_cols) \
                                     .pivot("lab_name") \
                                     .agg(f.min('lab_value_num').alias("min"),
                                         f.max('lab_value_num').alias("max")).orderBy("life_support_start_time")

rows = cohort_labs_wide_summary.count()
print(f"DataFrame Rows count : {rows}")

cohort_labs_wide_summary.write.parquet("/project2/wparker/SIPA_data/cohort_labs_48_summary.parquet", mode="overwrite")

DataFrame Rows count : 10479
